<a href="https://colab.research.google.com/github/myPar/NSU_Practice/blob/dev/baseline/RNNbaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Prepare

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from pathlib import Path

import tensorflow as tf
from tf.keras.layers import SimpleRNN, Input
from tf.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = Path('/content/drive/MyDrive/Colab Notebooks/data.txt')
df = pd.read_csv(filepath_or_buffer=file_path, sep=' ', header=0)

df.head

<bound method NDFrame.head of              Date      Time  qo_lc[m3/d]  qw_lc[m3/d]  qg_lc[m3/d]  \
0     07-May-2014  10:13:26      109.280    -0.312890       3810.3   
1     07-May-2014  10:14:26      113.460    -0.437350       3774.0   
2     07-May-2014  10:15:26      131.630    -0.533550       3892.5   
3     07-May-2014  10:16:26      140.670     0.160910       4043.1   
4     07-May-2014  10:17:26      150.020    -1.145600       4134.6   
...           ...       ...          ...          ...          ...   
1264  08-May-2014  07:34:35       98.744     0.310070       3174.5   
1265  08-May-2014  07:35:35       72.045     0.162760       3134.1   
1266  08-May-2014  07:36:35       87.370    -0.125770       3340.4   
1267  08-May-2014  07:37:35       32.121     0.021522       2834.1   
1268  08-May-2014  07:38:35       35.914     0.228020       2612.4   

      qo_sc[Sm3/d]  qw_sc[Sm3/d]  qg_sc[Sm3/d]  qo_scnp[Sm3/d]  \
0          108.430     -0.314870       92700.0         108.430 

In [ ]:
print(df.columns)

Index(['Date', 'Time', 'qo_lc[m3/d]', 'qw_lc[m3/d]', 'qg_lc[m3/d]',
       'qo_sc[Sm3/d]', 'qw_sc[Sm3/d]', 'qg_sc[Sm3/d]', 'qo_scnp[Sm3/d]',
       'qw_scnp[Sm3/d]', 'qg_scnp[Sm3/d]', 'Fo_lc[%]', 'Fw_lc[%]', 'Fg_lc[%]',
       'WLR[%]', 'GVF[%]', 'GLR[m3/m3]', 'BSW[%]', 'GOR[Sm3/Sm3]',
       'GOR1[Sm3/Sm3]', 'mo_lc[kg/d]', 'mw_lc[kg/d]', 'mg_lc[kg/d]',
       'm_lc[kg/d]', 'mo_sc[kg/d]', 'mw_sc[kg/d]', 'mg_sc[kg/d]',
       'Mu_o_lc[cP]', 'Mu_l_lc[cP]', 'Do_lc[g/cm3]', 'Dw_lc[g/cm3]',
       'Dg_lc[g/cm3]', 'Dm_lc[g/cm3]', 'Dl_lc[g/cm3]', 'bo[Sm3/m3]',
       'bw[Sm3/m3]', 'bg[Sm3/m3]', 'Z', 'Rst[Sm3/Sm3]', 'Rwst[Sm3/Sm3]',
       'rgmp[Sm3/Sm3]', 'N32[cps]', 'N81[cps]', 'N356[cps]', 'NTotal[cps]',
       'DeadTime[s]', 'SampleTime[s]', 'DPV[mbar]', 'PL[bara]', 'TL[DegC]',
       'TAMB[DegC]', 'Dyn_DP[mbar]', 'OperatingPointLE[1/m]',
       'OperatingPointHE[1/m]', 'OilPointLE[1/m]', 'OilPointHE[1/m]',
       'WaterPointLE[1/m]', 'WaterPointHE[1/m]', 'GasPointLE[1/m]',
       'GasPoin

In [ ]:
selected_rows = ['DPV[mbar]', 'PL[bara]', 'qg_sc[Sm3/d]', 'qo_sc[Sm3/d]', 'TL[DegC]']
outputs = ['qg_sc[Sm3/d]', 'qo_sc[Sm3/d]']
inputs = np.setdiff1d(selected_rows, outputs)

df = df[selected_rows]

print(df.head)
print("inputs: " + str(inputs))
print("outputs: " + str(outputs))

<bound method NDFrame.head of       DPV[mbar]  PL[bara]  qg_sc[Sm3/d]  qo_sc[Sm3/d]  TL[DegC]
0       1448.70    21.481       92700.0       108.430   -2.5452
1       1462.10    21.473       91895.0       112.580   -2.5323
2       1592.30    21.672       95860.0       130.550   -2.3931
3       1703.30    21.926      100690.0       139.440   -2.0451
4       1821.70    22.196      104290.0       148.640   -1.8009
...         ...       ...           ...           ...       ...
1264    1147.10    23.997       87402.0        97.703   -3.3542
1265    1090.50    23.919       85594.0        71.288   -3.2890
1266    1279.10    24.477       93660.0        86.386   -3.2191
1267     976.62    23.808       76564.0        31.789   -3.2819
1268        NaN       NaN       69609.0        35.567       NaN

[1269 rows x 5 columns]>

In [ ]:
# check categorical attributes
cardinality_hold = 0.15
expected_col_size = 1269

categorical_attributes = []

def check_attributes_info():
  for column in df.columns:
    col_data = df[column].to_numpy()
    size = len(col_data)
    
    assert size == expected_col_size
    cleared_data = col_data[~np.isnan(col_data)]
    
    card = len(np.unique(cleared_data))  # cardinality without Nan items
    skip_count = size - len(cleared_data)

    print(column + ": card=" + str(card) + "; skip: percent=" + str(skip_count * 100 / size) + ", count=" + str(skip_count))
    
    if card / size < cardinality_hold:
      categorical_attributes.append(card)

check_attributes_info()

if len(categorical_attributes) == 0:
  print("no categorical attributes")
else:
  print(categorical_attributes)

DPV[mbar]: card=1163; skip: percent=0.07880220646178093, count=1
PL[bara]: card=1042; skip: percent=0.07880220646178093, count=1
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
TL[DegC]: card=1214; skip: percent=0.07880220646178093, count=1
no categorical attributes


In [ ]:
# supplement data
for column in df.columns:
  data =  np.array(df[column].to_numpy(), dtype=np.float64)

  for i in range(len(data)):
    if str(data[i]) == 'nan':
      data[i] = np.mean(data[~np.isnan(data)])
  df.drop([column], axis=1, inplace=True)
  df.insert(0, column, data)


In [ ]:
check_attributes_info()

TL[DegC]: card=1215; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
PL[bara]: card=1043; skip: percent=0.0, count=0
DPV[mbar]: card=1164; skip: percent=0.0, count=0


## Prepare data

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3)
print(train_df.head)
print(test_df.head)

train_input_df = train_df[inputs]
train_output_df = train_df[outputs]

test_input_df = test_df[inputs]
test_output_df = test_df[outputs]


## Create model

In [ ]:
# model parameters
batch_size = 60
timesteps = 60
input_size = len(inputs)
output_seze = len(outputs)
lr = 0.001

mse = tf.keras.losses.MeanSquaredError()

model = keras.Sequential()
model.add(Input((timesteps, imput_size)))
model.add(SimpleRNN(units=120, activation='tanh', retutn_sequence=True))
model.add(SimpleRNN(units=120, activation='tanh', retutn_sequence=True))
model.add(SimpleRNN(units=2, activation='tanh', retutn_sequence=True))
model.summary()

In [ ]:
model.compile(loss=mse, metrics=[mse], optimizer=Adam(learning_rate=lr))